In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 49000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:50000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_49k_50k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6782716915781505024-2
Page load happened
49000 https://kw.com/agent/UPA-6782716915781505024-2
***********************************
https://kw.com/agent/UPA-6587385251019313156-5
Page load happened
49001 https://kw.com/agent/UPA-6587385251019313156-5
***********************************
https://kw.com/agent/UPA-6587385250259079168-6
Page load happened
49002 https://kw.com/agent/UPA-6587385250259079168-6
***********************************
https://kw.com/agent/UPA-6587384978184663045-8
Page load happened
49003 https://kw.com/agent/UPA-6587384978184663045-8
***********************************
https://kw.com/agent/UPA-6760340605431431168-6
Page load happened
49004 https://kw.com/agent/UPA-6760340605431431168-6
***********************************
https://kw.com/agent/UPA-6587385246429265924-0
Page load happened
49005 https://kw.com/agent/UPA-6587385246429265924-0
***********************************
https://kw.com/agent/UPA-6587385088285888514-9
Page load happened
4900

Page load happened
49053 https://kw.com/agent/UPA-6587385339191062534-6
***********************************
https://kw.com/agent/UPA-6587385281882030080-2
Page load happened
49054 https://kw.com/agent/UPA-6587385281882030080-2
***********************************
https://kw.com/agent/UPA-6838480244641718272-1
Page load happened
49055 https://kw.com/agent/UPA-6838480244641718272-1
***********************************
https://kw.com/agent/UPA-6587385152072908803-9
Page load happened
49056 https://kw.com/agent/UPA-6587385152072908803-9
***********************************
https://kw.com/agent/UPA-6587385160485658626-0
Page load happened
49057 https://kw.com/agent/UPA-6587385160485658626-0
***********************************
https://kw.com/agent/UPA-6592263194613387269-4
Page load happened
49058 https://kw.com/agent/UPA-6592263194613387269-4
***********************************
https://kw.com/agent/UPA-6587385103832268807-8
Page load happened
49059 https://kw.com/agent/UPA-6587385103832268807-

Page load happened
49106 https://kw.com/agent/UPA-6631251446429323265-8
***********************************
https://kw.com/agent/UPA-6587385438464356353-4
Page load happened
49107 https://kw.com/agent/UPA-6587385438464356353-4
***********************************
https://kw.com/agent/UPA-6587385402495320067-6
Page load happened
49108 https://kw.com/agent/UPA-6587385402495320067-6
***********************************
https://kw.com/agent/UPA-6798359477117988864-0
Page load happened
49109 https://kw.com/agent/UPA-6798359477117988864-0
***********************************
https://kw.com/agent/UPA-6608309913605607428-4
Page load happened
49110 https://kw.com/agent/UPA-6608309913605607428-4
***********************************
https://kw.com/agent/UPA-6592229576722497536-9
Page load happened
49111 https://kw.com/agent/UPA-6592229576722497536-9
***********************************
https://kw.com/agent/UPA-6709560401267060736-2
Page load happened
49112 https://kw.com/agent/UPA-6709560401267060736-

Page load happened
49159 https://kw.com/agent/UPA-6587385371716755461-9
***********************************
https://kw.com/agent/UPA-6617821703887130624-8
Timeout happened no page load
49160 https://kw.com/agent/UPA-6617821703887130624-8
***********************************
https://kw.com/agent/UPA-6587385397462528006-7
Page load happened
49161 https://kw.com/agent/UPA-6587385397462528006-7
***********************************
https://kw.com/agent/UPA-6587385187729981441-2
Page load happened
49162 https://kw.com/agent/UPA-6587385187729981441-2
***********************************
https://kw.com/agent/UPA-6709530090694688768-5
Page load happened
49163 https://kw.com/agent/UPA-6709530090694688768-5
***********************************
https://kw.com/agent/UPA-6597235333125251075-8
Page load happened
49164 https://kw.com/agent/UPA-6597235333125251075-8
***********************************
https://kw.com/agent/UPA-6592994385047547906-8
Page load happened
49165 https://kw.com/agent/UPA-659299438

Page load happened
49212 https://kw.com/agent/UPA-6587385414339784705-3
***********************************
https://kw.com/agent/UPA-6592976346321981442-2
Page load happened
49213 https://kw.com/agent/UPA-6592976346321981442-2
***********************************
https://kw.com/agent/UPA-6732073314914426880-7
Page load happened
49214 https://kw.com/agent/UPA-6732073314914426880-7
***********************************
https://kw.com/agent/UPA-6587385077969186821-3
Page load happened
49215 https://kw.com/agent/UPA-6587385077969186821-3
***********************************
https://kw.com/agent/UPA-6763162708857561088-7
Page load happened
49216 https://kw.com/agent/UPA-6763162708857561088-7
***********************************
https://kw.com/agent/UPA-6587385184040046595-8
Page load happened
49217 https://kw.com/agent/UPA-6587385184040046595-8
***********************************
https://kw.com/agent/UPA-6643176992101031936-8
Page load happened
49218 https://kw.com/agent/UPA-6643176992101031936-

...

Timeout happened no page load
49742 https://kw.com/agent/UPA-6592237257279897602-1
***********************************
https://kw.com/agent/UPA-6587385023656787973-9
Page load happened
49743 https://kw.com/agent/UPA-6587385023656787973-9
***********************************
https://kw.com/agent/UPA-6764960869668335616-8
Page load happened
49744 https://kw.com/agent/UPA-6764960869668335616-8
***********************************
https://kw.com/agent/UPA-6587385237026148356-2
Page load happened
49745 https://kw.com/agent/UPA-6587385237026148356-2
***********************************
https://kw.com/agent/UPA-6587385058881159173-4
Page load happened
49746 https://kw.com/agent/UPA-6587385058881159173-4
***********************************
https://kw.com/agent/UPA-6587385370478149637-0
Page load happened
49747 https://kw.com/agent/UPA-6587385370478149637-0
***********************************
https://kw.com/agent/UPA-6587385385713803272-8
Page load happened
49748 https://kw.com/agent/UPA-658738538

Timeout happened no page load
49795 https://kw.com/agent/UPA-6776194146605260800-1
***********************************
https://kw.com/agent/UPA-6587385318536601604-1
Page load happened
49796 https://kw.com/agent/UPA-6587385318536601604-1
***********************************
https://kw.com/agent/UPA-6587385091724644353-8
Page load happened
49797 https://kw.com/agent/UPA-6587385091724644353-8
***********************************
https://kw.com/agent/UPA-6587385144154238979-3
Page load happened
49798 https://kw.com/agent/UPA-6587385144154238979-3
***********************************
https://kw.com/agent/UPA-6587385264279105538-4
Page load happened
49799 https://kw.com/agent/UPA-6587385264279105538-4
***********************************
https://kw.com/agent/UPA-6587385448023998469-9
need to close driver
new driver initialize****************
Page load happened
49800 https://kw.com/agent/UPA-6587385448023998469-9
***********************************
https://kw.com/agent/UPA-6778052649270251520-7


Page load happened
49848 https://kw.com/agent/UPA-6587385400008896512-0
***********************************
https://kw.com/agent/UPA-6587385295173382148-9
Page load happened
49849 https://kw.com/agent/UPA-6587385295173382148-9
***********************************
https://kw.com/agent/UPA-6773370248991264768-3
Page load happened
49850 https://kw.com/agent/UPA-6773370248991264768-3
***********************************
https://kw.com/agent/UPA-6587385330075975687-6
Page load happened
49851 https://kw.com/agent/UPA-6587385330075975687-6
***********************************
https://kw.com/agent/UPA-6587385189731512321-4
Page load happened
49852 https://kw.com/agent/UPA-6587385189731512321-4
***********************************
https://kw.com/agent/UPA-6587385042575040514-0
Page load happened
49853 https://kw.com/agent/UPA-6587385042575040514-0
***********************************
https://kw.com/agent/UPA-6587385185131728900-8
Page load happened
49854 https://kw.com/agent/UPA-6587385185131728900-

Page load happened
49901 https://kw.com/agent/UPA-6587385354185641993-9
***********************************
https://kw.com/agent/UPA-6587385247618535429-0
Page load happened
49902 https://kw.com/agent/UPA-6587385247618535429-0
***********************************
https://kw.com/agent/UPA-6587385413019144194-9
Page load happened
49903 https://kw.com/agent/UPA-6587385413019144194-9
***********************************
https://kw.com/agent/UPA-6845936737259323392-8
Page load happened
49904 https://kw.com/agent/UPA-6845936737259323392-8
***********************************
https://kw.com/agent/UPA-6674452988969664512-0
Page load happened
49905 https://kw.com/agent/UPA-6674452988969664512-0
***********************************
https://kw.com/agent/UPA-6587385109169528832-3
Page load happened
49906 https://kw.com/agent/UPA-6587385109169528832-3
***********************************
https://kw.com/agent/UPA-6587385244267220997-8
Page load happened
49907 https://kw.com/agent/UPA-6587385244267220997-

Page load happened
49954 https://kw.com/agent/UPA-6587385020490674181-3
***********************************
https://kw.com/agent/UPA-6587385359936573443-8
Page load happened
49955 https://kw.com/agent/UPA-6587385359936573443-8
***********************************
https://kw.com/agent/UPA-6587385291433807880-2
Page load happened
49956 https://kw.com/agent/UPA-6587385291433807880-2
***********************************
https://kw.com/agent/UPA-6587384994362839043-4
Page load happened
49957 https://kw.com/agent/UPA-6587384994362839043-4
***********************************
https://kw.com/agent/UPA-6587385187465740292-7
Page load happened
49958 https://kw.com/agent/UPA-6587385187465740292-7
***********************************
https://kw.com/agent/UPA-6587385425929113600-0
Page load happened
49959 https://kw.com/agent/UPA-6587385425929113600-0
***********************************
https://kw.com/agent/UPA-6587385215020339207-9
Page load happened
49960 https://kw.com/agent/UPA-6587385215020339207-